# QC-Py-26 - LLM Trading Signals

> **Large Language Models pour l'analyse de marche et la generation de signaux**
> Duree: 90 minutes | Niveau: Avance | Python + QuantConnect

---

## Objectifs d'Apprentissage

A la fin de ce notebook, vous serez capable de :

1. Comprendre les **capacites et limites** des LLMs pour le trading
2. Maitriser le **prompt engineering** pour l'analyse financiere
3. Utiliser l'API **OpenAI** (GPT-4) pour l'analyse de news
4. Utiliser l'API **Anthropic** (Claude) pour le raisonnement structure
5. Implementer un systeme de **sentiment analysis** base LLM
6. Combiner signaux LLM avec **indicateurs techniques**
7. Gerer les **couts API** et la **latence**
8. Integrer les LLMs dans un **Alpha Model** QuantConnect

## Prerequisites

- Notebooks QC-Py-01 a 25 completes
- Comprehension du sentiment analysis (QC-Py-17)
- Cle API OpenAI et/ou Anthropic
- Notions de prompt engineering

## Structure du Notebook

| Partie | Sujet | Duree |
|--------|-------|-------|
| 1 | LLMs pour la Finance : Opportunites et Limites | 10 min |
| 2 | Prompt Engineering pour le Trading | 15 min |
| 3 | Analyse de Sentiment avec GPT-4 | 20 min |
| 4 | Raisonnement Structure avec Claude | 15 min |
| 5 | Systeme Hybride LLM + Indicateurs | 15 min |
| 6 | Integration QuantConnect | 15 min |

---

## Introduction : LLMs dans le Trading

### Revolution 2023-2026

L'emergence des LLMs a transforme l'analyse financiere :

| Annee | Developpement | Impact Trading |
|-------|---------------|----------------|
| 2022 | ChatGPT lance | Debut des experimentations |
| 2023 | GPT-4, Claude 2 | Premiers hedgefunds adoptent |
| 2024 | Claude 3.5, GPT-4o | Integration production |
| 2025 | Claude Opus 4, GPT-5 | Agents trading autonomes |
| 2026 | Modeles specialises finance | Mainstream dans quant finance |

### Cas d'Usage

```
                        LLMs en Trading
                              |
       +----------------------+----------------------+
       |                      |                      |
  Analyse de News        Interpretation        Generation
  - Sentiment            - Earnings calls      - Reports
  - Event extraction     - SEC filings         - Strategies
  - Summarization        - Economic data       - Code
```

### Avantages vs Limites

| Avantages | Limites |
|-----------|---------||
| Comprehension du langage naturel | Couts API eleves |
| Raisonnement contextuel | Latence (100ms-2s) |
| Generalisation zero-shot | Hallucinations possibles |
| Mise a jour des connaissances | Knowledge cutoff |
| Multi-tache sans retraining | Non deterministe |

In [ ]:
# Imports et configuration

import os
import json
import time
from typing import Dict, List, Optional, Tuple
from dataclasses import dataclass, field
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# API clients (installer si necessaire)
# pip install openai anthropic

try:
    import openai
    OPENAI_AVAILABLE = True
except ImportError:
    OPENAI_AVAILABLE = False
    print("OpenAI not installed. Run: pip install openai")

try:
    import anthropic
    ANTHROPIC_AVAILABLE = True
except ImportError:
    ANTHROPIC_AVAILABLE = False
    print("Anthropic not installed. Run: pip install anthropic")

# Configuration matplotlib
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

print(f"OpenAI disponible: {OPENAI_AVAILABLE}")
print(f"Anthropic disponible: {ANTHROPIC_AVAILABLE}")
print("\nNote: Ce notebook necessite des cles API valides pour les appels reels.")
print("Les exemples utilisent des reponses simulees si les APIs ne sont pas configurees.")

In [ ]:
# Configuration des APIs (utiliser .env en production)

@dataclass
class LLMConfig:
    """Configuration pour les LLMs."""
    openai_api_key: Optional[str] = None
    anthropic_api_key: Optional[str] = None
    openai_model: str = "gpt-4o-mini"  # Economique pour tests
    claude_model: str = "claude-3-5-sonnet-20241022"  # Equilibre cout/performance
    max_tokens: int = 1000
    temperature: float = 0.3  # Faible pour coherence
    
    def __post_init__(self):
        # Charger depuis environnement si non specifie
        if self.openai_api_key is None:
            self.openai_api_key = os.environ.get('OPENAI_API_KEY')
        if self.anthropic_api_key is None:
            self.anthropic_api_key = os.environ.get('ANTHROPIC_API_KEY')


# Initialiser configuration
config = LLMConfig()

print("Configuration LLM:")
print(f"  OpenAI Key: {'***' + config.openai_api_key[-4:] if config.openai_api_key else 'Non configuree'}")
print(f"  Anthropic Key: {'***' + config.anthropic_api_key[-4:] if config.anthropic_api_key else 'Non configuree'}")
print(f"  OpenAI Model: {config.openai_model}")
print(f"  Claude Model: {config.claude_model}")

---

## Partie 1 : Couts et Gestion des APIs (10 min)

### Tarification (Janvier 2026)

| Modele | Input (1M tokens) | Output (1M tokens) | Latence |
|--------|-------------------|--------------------|---------|
| GPT-4o | $2.50 | $10.00 | 500ms |
| GPT-4o-mini | $0.15 | $0.60 | 200ms |
| Claude 3.5 Sonnet | $3.00 | $15.00 | 400ms |
| Claude 3 Haiku | $0.25 | $1.25 | 150ms |

### Estimation des Couts

Pour une strategie quotidienne analysant 10 articles par actif, 50 actifs :

```
Tokens par analyse: ~500 input + ~200 output
Analyses par jour: 10 * 50 = 500
Tokens par jour: 500 * 700 = 350,000

Cout mensuel (GPT-4o-mini):
  Input: 350K * 30 * $0.15/1M = $1.58
  Output: 100K * 30 * $0.60/1M = $1.80
  Total: ~$3.40/mois
```

In [ ]:
# Gestionnaire de couts et rate limiting

@dataclass
class CostTracker:
    """
    Suivi des couts et usage des APIs LLM.
    """
    daily_budget: float = 1.0  # $1/jour par defaut
    input_tokens_used: int = 0
    output_tokens_used: int = 0
    requests_made: int = 0
    
    # Tarification (a jour janvier 2026)
    PRICING = {
        'gpt-4o': {'input': 2.50 / 1_000_000, 'output': 10.00 / 1_000_000},
        'gpt-4o-mini': {'input': 0.15 / 1_000_000, 'output': 0.60 / 1_000_000},
        'claude-3-5-sonnet-20241022': {'input': 3.00 / 1_000_000, 'output': 15.00 / 1_000_000},
        'claude-3-haiku-20240307': {'input': 0.25 / 1_000_000, 'output': 1.25 / 1_000_000},
    }
    
    def estimate_cost(self, model: str, input_tokens: int, output_tokens: int) -> float:
        """Estime le cout d'une requete."""
        if model not in self.PRICING:
            return 0.0
        
        pricing = self.PRICING[model]
        return (input_tokens * pricing['input'] + output_tokens * pricing['output'])
    
    def record_usage(self, model: str, input_tokens: int, output_tokens: int):
        """Enregistre l'usage."""
        self.input_tokens_used += input_tokens
        self.output_tokens_used += output_tokens
        self.requests_made += 1
    
    def get_total_cost(self, model: str) -> float:
        """Calcule le cout total."""
        return self.estimate_cost(model, self.input_tokens_used, self.output_tokens_used)
    
    def within_budget(self, model: str) -> bool:
        """Verifie si dans le budget."""
        return self.get_total_cost(model) < self.daily_budget
    
    def report(self, model: str) -> str:
        """Genere un rapport d'usage."""
        cost = self.get_total_cost(model)
        return (
            f"Usage Report:\n"
            f"  Requests: {self.requests_made}\n"
            f"  Input tokens: {self.input_tokens_used:,}\n"
            f"  Output tokens: {self.output_tokens_used:,}\n"
            f"  Estimated cost: ${cost:.4f}\n"
            f"  Budget remaining: ${self.daily_budget - cost:.4f}"
        )


# Demonstration
tracker = CostTracker(daily_budget=5.0)

# Simuler quelques requetes
for i in range(10):
    tracker.record_usage('gpt-4o-mini', input_tokens=500, output_tokens=200)

print(tracker.report('gpt-4o-mini'))

---

## Partie 2 : Prompt Engineering pour le Trading (15 min)

### Principes Cles

| Principe | Description | Exemple |
|----------|-------------|--------|
| **Specificite** | Instructions precises | "Analyse le sentiment: BULLISH, BEARISH, ou NEUTRAL" |
| **Structure** | Format de sortie defini | JSON avec champs obligatoires |
| **Contexte** | Information de fond | "Tu es un analyste quantitatif senior" |
| **Exemples** | Few-shot learning | 2-3 exemples annotes |
| **Contraintes** | Limites claires | "Reponds en moins de 100 mots" |

### Template de Prompt

```
[ROLE]
Tu es un analyste financier quantitatif...

[TASK]
Analyse le texte suivant et determine...

[FORMAT]
Reponds en JSON avec les champs:
- sentiment: BULLISH | BEARISH | NEUTRAL
- confidence: 0.0 a 1.0
- reasoning: explication breve

[EXAMPLES]
Exemple 1: "..." -> {...}

[INPUT]
{texte a analyser}
```

In [ ]:
# Templates de prompts pour le trading

class TradingPrompts:
    """
    Collection de prompts optimises pour l'analyse de trading.
    """
    
    SENTIMENT_ANALYSIS = """
You are a senior quantitative analyst at a hedge fund. Your task is to analyze 
financial news and determine the market sentiment for a specific stock.

RULES:
1. Focus on facts that could impact stock price in the next 1-5 days
2. Ignore general market commentary unless directly relevant
3. Weight recent information more heavily
4. Consider both explicit statements and implicit implications

OUTPUT FORMAT (JSON only, no other text):
{{
    "sentiment": "BULLISH" | "BEARISH" | "NEUTRAL",
    "confidence": <float 0.0-1.0>,
    "key_factors": ["factor1", "factor2"],
    "price_impact": "HIGH" | "MEDIUM" | "LOW",
    "time_horizon": "<number> days"
}}

STOCK: {symbol}
NEWS:
{news_text}
"""

    EARNINGS_ANALYSIS = """
You are analyzing an earnings report for trading signals.

Focus on:
1. EPS vs expectations (beat/miss magnitude)
2. Revenue growth and guidance
3. Margin trends
4. Forward guidance tone
5. Key business metrics mentioned

OUTPUT FORMAT (JSON only):
{{
    "overall_signal": "BUY" | "SELL" | "HOLD",
    "earnings_surprise": <percentage>,
    "guidance_tone": "POSITIVE" | "NEGATIVE" | "NEUTRAL",
    "key_metrics": {{}},
    "risk_factors": [],
    "confidence": <float 0.0-1.0>
}}

COMPANY: {symbol}
EARNINGS REPORT:
{earnings_text}
"""

    MULTI_ASSET_SUMMARY = """
You are summarizing market conditions for portfolio management.

Analyze the following market data and news across multiple assets.
Identify:
1. Overall market sentiment
2. Sector rotations or themes
3. Risk-on vs risk-off indicators
4. Key events in next 7 days

OUTPUT FORMAT (JSON only):
{{
    "market_regime": "RISK_ON" | "RISK_OFF" | "NEUTRAL",
    "sector_preferences": ["sector1", "sector2"],
    "sectors_to_avoid": ["sector3"],
    "key_events": [{"date": "...", "event": "...", "impact": "..."}],
    "overall_confidence": <float 0.0-1.0>
}}

MARKET DATA:
{market_data}
"""

    TECHNICAL_INTERPRETATION = """
You are interpreting technical indicators for trading decisions.

Given the following technical data, provide a trading recommendation.
Consider indicator confluence and potential false signals.

OUTPUT FORMAT (JSON only):
{{
    "direction": "LONG" | "SHORT" | "FLAT",
    "strength": "STRONG" | "MODERATE" | "WEAK",
    "entry_condition": "description",
    "stop_loss": "description or price",
    "take_profit": "description or price",
    "timeframe": "INTRADAY" | "SWING" | "POSITION",
    "confidence": <float 0.0-1.0>
}}

SYMBOL: {symbol}
TECHNICAL DATA:
{technical_data}
"""


# Demonstration
print("Templates de Prompts Disponibles:")
print("  1. SENTIMENT_ANALYSIS - Analyse de sentiment de news")
print("  2. EARNINGS_ANALYSIS - Analyse de resultats")
print("  3. MULTI_ASSET_SUMMARY - Resume multi-actifs")
print("  4. TECHNICAL_INTERPRETATION - Interpretation technique")

# Exemple de formatage
example_prompt = TradingPrompts.SENTIMENT_ANALYSIS.format(
    symbol="AAPL",
    news_text="Apple announces record iPhone sales in Q4, beating analyst expectations by 15%..."
)
print(f"\nExemple de prompt formate ({len(example_prompt)} caracteres):")
print(example_prompt[:500] + "...")

---

## Partie 3 : Analyse de Sentiment avec GPT-4 (20 min)

### Architecture

```
News Feed
    |
    v
Preprocessing
  - Filtrage
  - Chunking
    |
    v
GPT-4 API
  - Prompt template
  - JSON parsing
    |
    v
Signal Processing
  - Aggregation
  - Confidence weighting
    |
    v
Trading Signal
```

In [ ]:
# Client LLM avec fallback simule

class LLMClient:
    """
    Client unifie pour OpenAI et Anthropic avec fallback simule.
    """
    
    def __init__(self, config: LLMConfig):
        self.config = config
        self.cost_tracker = CostTracker()
        
        # Initialiser clients si disponibles
        self.openai_client = None
        self.anthropic_client = None
        
        if OPENAI_AVAILABLE and config.openai_api_key:
            self.openai_client = openai.OpenAI(api_key=config.openai_api_key)
        
        if ANTHROPIC_AVAILABLE and config.anthropic_api_key:
            self.anthropic_client = anthropic.Anthropic(api_key=config.anthropic_api_key)
    
    def _simulate_response(self, prompt: str, task_type: str) -> Dict:
        """
        Simule une reponse LLM pour demonstration.
        """
        # Analyser le prompt pour determiner le sentiment simule
        positive_words = ['beat', 'record', 'growth', 'exceed', 'strong', 'bullish']
        negative_words = ['miss', 'decline', 'weak', 'loss', 'bearish', 'warning']
        
        prompt_lower = prompt.lower()
        pos_count = sum(1 for w in positive_words if w in prompt_lower)
        neg_count = sum(1 for w in negative_words if w in prompt_lower)
        
        if pos_count > neg_count:
            sentiment = "BULLISH"
            confidence = min(0.5 + pos_count * 0.1, 0.9)
        elif neg_count > pos_count:
            sentiment = "BEARISH"
            confidence = min(0.5 + neg_count * 0.1, 0.9)
        else:
            sentiment = "NEUTRAL"
            confidence = 0.5
        
        return {
            "sentiment": sentiment,
            "confidence": confidence,
            "key_factors": ["simulated_factor_1", "simulated_factor_2"],
            "price_impact": "MEDIUM",
            "time_horizon": "5 days",
            "_simulated": True
        }
    
    def analyze_with_openai(self, prompt: str) -> Tuple[Dict, Dict]:
        """
        Analyse avec OpenAI GPT-4.
        
        Returns:
            tuple: (result_dict, usage_dict)
        """
        if self.openai_client is None:
            result = self._simulate_response(prompt, "sentiment")
            usage = {"input_tokens": len(prompt) // 4, "output_tokens": 100, "simulated": True}
            return result, usage
        
        try:
            response = self.openai_client.chat.completions.create(
                model=self.config.openai_model,
                messages=[
                    {"role": "system", "content": "You are a financial analyst. Always respond with valid JSON."},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=self.config.max_tokens,
                temperature=self.config.temperature,
                response_format={"type": "json_object"}
            )
            
            content = response.choices[0].message.content
            result = json.loads(content)
            
            usage = {
                "input_tokens": response.usage.prompt_tokens,
                "output_tokens": response.usage.completion_tokens,
                "simulated": False
            }
            
            self.cost_tracker.record_usage(
                self.config.openai_model,
                usage["input_tokens"],
                usage["output_tokens"]
            )
            
            return result, usage
            
        except Exception as e:
            print(f"OpenAI error: {e}")
            result = self._simulate_response(prompt, "sentiment")
            result["_error"] = str(e)
            return result, {"simulated": True, "error": str(e)}
    
    def analyze_with_claude(self, prompt: str) -> Tuple[Dict, Dict]:
        """
        Analyse avec Anthropic Claude.
        """
        if self.anthropic_client is None:
            result = self._simulate_response(prompt, "sentiment")
            usage = {"input_tokens": len(prompt) // 4, "output_tokens": 100, "simulated": True}
            return result, usage
        
        try:
            response = self.anthropic_client.messages.create(
                model=self.config.claude_model,
                max_tokens=self.config.max_tokens,
                messages=[
                    {"role": "user", "content": prompt}
                ]
            )
            
            content = response.content[0].text
            
            # Extraire JSON du contenu
            try:
                # Chercher JSON dans la reponse
                start = content.find('{')
                end = content.rfind('}') + 1
                if start != -1 and end > start:
                    result = json.loads(content[start:end])
                else:
                    result = {"raw_response": content}
            except json.JSONDecodeError:
                result = {"raw_response": content}
            
            usage = {
                "input_tokens": response.usage.input_tokens,
                "output_tokens": response.usage.output_tokens,
                "simulated": False
            }
            
            self.cost_tracker.record_usage(
                self.config.claude_model,
                usage["input_tokens"],
                usage["output_tokens"]
            )
            
            return result, usage
            
        except Exception as e:
            print(f"Claude error: {e}")
            result = self._simulate_response(prompt, "sentiment")
            result["_error"] = str(e)
            return result, {"simulated": True, "error": str(e)}


# Initialiser client
llm_client = LLMClient(config)
print("LLM Client initialise")

In [ ]:
# Analyseur de sentiment complet

@dataclass
class SentimentResult:
    """Resultat d'analyse de sentiment."""
    symbol: str
    sentiment: str  # BULLISH, BEARISH, NEUTRAL
    confidence: float
    key_factors: List[str]
    price_impact: str
    time_horizon: str
    source: str  # openai, claude, simulated
    timestamp: datetime = field(default_factory=datetime.now)
    
    def to_signal(self) -> float:
        """Convertit en signal numerique [-1, 1]."""
        base = {
            "BULLISH": 1.0,
            "BEARISH": -1.0,
            "NEUTRAL": 0.0
        }.get(self.sentiment, 0.0)
        
        return base * self.confidence


class NewsSentimentAnalyzer:
    """
    Analyseur de sentiment pour news financieres.
    """
    
    def __init__(self, llm_client: LLMClient, provider: str = "openai"):
        self.client = llm_client
        self.provider = provider
        self.results_cache = {}
    
    def analyze(self, symbol: str, news_text: str) -> SentimentResult:
        """
        Analyse le sentiment d'un texte de news.
        """
        # Construire prompt
        prompt = TradingPrompts.SENTIMENT_ANALYSIS.format(
            symbol=symbol,
            news_text=news_text[:2000]  # Limiter la taille
        )
        
        # Appeler LLM
        if self.provider == "openai":
            result, usage = self.client.analyze_with_openai(prompt)
        else:
            result, usage = self.client.analyze_with_claude(prompt)
        
        # Parser resultat
        sentiment_result = SentimentResult(
            symbol=symbol,
            sentiment=result.get("sentiment", "NEUTRAL"),
            confidence=result.get("confidence", 0.5),
            key_factors=result.get("key_factors", []),
            price_impact=result.get("price_impact", "MEDIUM"),
            time_horizon=result.get("time_horizon", "5 days"),
            source="simulated" if usage.get("simulated") else self.provider
        )
        
        return sentiment_result
    
    def analyze_batch(self, symbol: str, news_list: List[str]) -> List[SentimentResult]:
        """
        Analyse un batch de news.
        """
        results = []
        for news in news_list:
            result = self.analyze(symbol, news)
            results.append(result)
        return results
    
    def aggregate_signals(self, results: List[SentimentResult]) -> Tuple[float, float]:
        """
        Agrege plusieurs signaux en un signal unique.
        
        Returns:
            tuple: (signal_aggrege, confiance_moyenne)
        """
        if not results:
            return 0.0, 0.0
        
        signals = [r.to_signal() for r in results]
        confidences = [r.confidence for r in results]
        
        # Moyenne ponderee par confiance
        total_weight = sum(confidences)
        if total_weight == 0:
            return 0.0, 0.0
        
        weighted_signal = sum(s * c for s, c in zip(signals, confidences)) / total_weight
        avg_confidence = np.mean(confidences)
        
        return weighted_signal, avg_confidence


# Demonstration
analyzer = NewsSentimentAnalyzer(llm_client, provider="openai")

# Exemples de news
sample_news = [
    "Apple reported record Q4 revenue of $95 billion, beating analyst expectations by 8%. iPhone sales grew 15% year-over-year.",
    "Apple faces supply chain challenges in China, potentially impacting holiday quarter shipments.",
    "Goldman Sachs upgrades Apple to Buy, citing strong services growth and AI opportunities."
]

print("Analyse de Sentiment - AAPL")
print("="*60)

results = []
for i, news in enumerate(sample_news):
    result = analyzer.analyze("AAPL", news)
    results.append(result)
    print(f"\nNews {i+1}:")
    print(f"  Sentiment: {result.sentiment}")
    print(f"  Confidence: {result.confidence:.2f}")
    print(f"  Signal: {result.to_signal():.2f}")
    print(f"  Source: {result.source}")

# Aggreger
agg_signal, agg_conf = analyzer.aggregate_signals(results)
print(f"\nSignal Agrege: {agg_signal:.2f} (confiance: {agg_conf:.2f})")

---

## Partie 4 : Raisonnement Structure avec Claude (15 min)

### Avantages de Claude pour l'Analyse

| Aspect | GPT-4 | Claude |
|--------|-------|--------|
| **Raisonnement** | Bon | Excellent |
| **Instructions longues** | Bon | Excellent |
| **Coherence JSON** | Excellent | Bon |
| **Cout** | Moyen | Similaire |
| **Contexte** | 128K | 200K |

### Chain-of-Thought pour Decisions Complexes

In [ ]:
# Analyseur avec Chain-of-Thought

class ChainOfThoughtAnalyzer:
    """
    Analyseur utilisant le raisonnement en chaine.
    """
    
    COT_PROMPT = """
You are a senior portfolio manager making trading decisions.

THINK STEP BY STEP:

Step 1: Identify the key facts
- List the most important data points
- Note any conflicting information

Step 2: Assess market context
- Current market regime (risk-on/risk-off)
- Sector trends
- Macro factors

Step 3: Evaluate technical setup
- Trend direction
- Support/resistance levels
- Momentum indicators

Step 4: Consider risks
- What could go wrong?
- Position sizing implications

Step 5: Make decision
- Clear action (BUY/SELL/HOLD)
- Entry/exit criteria
- Position size recommendation

FINAL OUTPUT (JSON):
{{
    "reasoning_steps": [
        {{"step": 1, "title": "Key Facts", "content": "..."}},
        {{"step": 2, "title": "Market Context", "content": "..."}},
        {{"step": 3, "title": "Technical Setup", "content": "..."}},
        {{"step": 4, "title": "Risk Assessment", "content": "..."}},
        {{"step": 5, "title": "Decision", "content": "..."}}
    ],
    "final_decision": {{
        "action": "BUY" | "SELL" | "HOLD",
        "conviction": "HIGH" | "MEDIUM" | "LOW",
        "position_size": "percentage of portfolio",
        "stop_loss": "description",
        "take_profit": "description",
        "time_horizon": "days/weeks"
    }}
}}

ANALYSIS CONTEXT:
Symbol: {symbol}

NEWS:
{news}

TECHNICAL DATA:
{technical}

PORTFOLIO CONTEXT:
{portfolio}
"""
    
    def __init__(self, llm_client: LLMClient):
        self.client = llm_client
    
    def analyze(self, symbol: str, news: str, technical: Dict, portfolio: Dict) -> Dict:
        """
        Analyse complete avec raisonnement structure.
        """
        prompt = self.COT_PROMPT.format(
            symbol=symbol,
            news=news,
            technical=json.dumps(technical, indent=2),
            portfolio=json.dumps(portfolio, indent=2)
        )
        
        # Utiliser Claude pour meilleur raisonnement
        result, usage = self.client.analyze_with_claude(prompt)
        
        return result


# Demonstration
cot_analyzer = ChainOfThoughtAnalyzer(llm_client)

# Donnees d'exemple
sample_technical = {
    "price": 185.50,
    "sma_20": 182.30,
    "sma_50": 178.90,
    "rsi_14": 62,
    "macd": {"macd": 2.1, "signal": 1.8, "histogram": 0.3},
    "volume_ratio": 1.2,
    "support": 180.00,
    "resistance": 190.00
}

sample_portfolio = {
    "current_position": "0%",
    "sector_exposure": {"technology": "25%"},
    "cash_available": "15%",
    "risk_budget": "moderate"
}

combined_news = "\n".join(sample_news)

print("Chain-of-Thought Analysis - AAPL")
print("="*60)

cot_result = cot_analyzer.analyze(
    "AAPL",
    combined_news,
    sample_technical,
    sample_portfolio
)

if "reasoning_steps" in cot_result:
    print("\nRaisonnement:")
    for step in cot_result.get("reasoning_steps", []):
        print(f"  Step {step.get('step', '?')}: {step.get('title', 'N/A')}")
        print(f"    {step.get('content', '')[:100]}...")

if "final_decision" in cot_result:
    decision = cot_result["final_decision"]
    print(f"\nDecision Finale:")
    print(f"  Action: {decision.get('action', 'N/A')}")
    print(f"  Conviction: {decision.get('conviction', 'N/A')}")
    print(f"  Position Size: {decision.get('position_size', 'N/A')}")
else:
    print(f"\nResultat brut: {json.dumps(cot_result, indent=2)[:500]}...")

---

## Partie 5 : Systeme Hybride LLM + Indicateurs (15 min)

### Architecture Hybride

```
             +----------------+
             |   LLM Signal   |
             | (Sentiment)    |
             +-------+--------+
                     |  0.4
                     v
+------------+  +----+----+  +------------+
| Technical  |  | Signal  |  | Risk       |
| Indicators +->| Fusion  |<-+ Filters    |
|   0.4      |  +---------+  |            |
+------------+       |       +------------+
                     v
             +-------+--------+
             | Final Signal   |
             | + Position     |
             +----------------+
```

### Avantages du Systeme Hybride

| Aspect | LLM Seul | Technique Seul | Hybride |
|--------|----------|----------------|-------|
| Comprehension news | Excellent | Aucune | Excellent |
| Objectivite | Moyenne | Excellente | Bonne |
| Latence | Haute | Faible | Moyenne |
| Cout | Eleve | Faible | Modere |
| Robustesse | Moyenne | Moyenne | Haute |

In [ ]:
# Systeme de fusion de signaux hybride

@dataclass
class HybridSignal:
    """Signal de trading hybride."""
    symbol: str
    llm_signal: float  # [-1, 1]
    llm_confidence: float
    technical_signal: float  # [-1, 1]
    technical_confidence: float
    fused_signal: float
    final_direction: str  # BUY, SELL, HOLD
    position_size: float  # 0 to 1
    timestamp: datetime = field(default_factory=datetime.now)


class HybridSignalSystem:
    """
    Systeme hybride combinant LLM et indicateurs techniques.
    """
    
    def __init__(
        self,
        llm_weight: float = 0.4,
        technical_weight: float = 0.4,
        agreement_bonus: float = 0.2,
        signal_threshold: float = 0.3
    ):
        self.llm_weight = llm_weight
        self.technical_weight = technical_weight
        self.agreement_bonus = agreement_bonus
        self.signal_threshold = signal_threshold
    
    def compute_technical_signal(self, data: Dict) -> Tuple[float, float]:
        """
        Calcule le signal technique a partir des indicateurs.
        
        Returns:
            tuple: (signal, confidence)
        """
        signals = []
        
        # Trend (SMA)
        if 'sma_20' in data and 'sma_50' in data and 'price' in data:
            price = data['price']
            sma20 = data['sma_20']
            sma50 = data['sma_50']
            
            # Price above both SMAs = bullish
            if price > sma20 > sma50:
                signals.append(1.0)
            elif price < sma20 < sma50:
                signals.append(-1.0)
            else:
                signals.append(0.0)
        
        # RSI
        if 'rsi_14' in data:
            rsi = data['rsi_14']
            if rsi > 70:
                signals.append(-0.5)  # Overbought
            elif rsi < 30:
                signals.append(0.5)   # Oversold
            elif rsi > 50:
                signals.append(0.25)
            else:
                signals.append(-0.25)
        
        # MACD
        if 'macd' in data:
            macd = data['macd']
            if isinstance(macd, dict):
                histogram = macd.get('histogram', 0)
                if histogram > 0:
                    signals.append(0.5)
                else:
                    signals.append(-0.5)
        
        # Volume
        if 'volume_ratio' in data:
            vol_ratio = data['volume_ratio']
            if vol_ratio > 1.5:
                # High volume confirms the move
                signals.append(0.25 if np.mean(signals) > 0 else -0.25)
        
        if not signals:
            return 0.0, 0.0
        
        signal = np.clip(np.mean(signals), -1, 1)
        
        # Confidence based on signal agreement
        signs = [np.sign(s) for s in signals if s != 0]
        if signs:
            agreement = abs(sum(signs)) / len(signs)
            confidence = 0.5 + 0.5 * agreement
        else:
            confidence = 0.5
        
        return signal, confidence
    
    def fuse_signals(
        self,
        llm_signal: float,
        llm_confidence: float,
        technical_signal: float,
        technical_confidence: float
    ) -> float:
        """
        Fusionne les signaux LLM et technique.
        """
        # Ponderation de base
        base_signal = (
            self.llm_weight * llm_signal * llm_confidence +
            self.technical_weight * technical_signal * technical_confidence
        )
        
        # Bonus si les deux signaux sont d'accord
        same_direction = np.sign(llm_signal) == np.sign(technical_signal)
        if same_direction and llm_signal != 0 and technical_signal != 0:
            agreement_strength = min(abs(llm_signal), abs(technical_signal))
            bonus = self.agreement_bonus * agreement_strength * np.sign(llm_signal)
            base_signal += bonus
        
        # Penalite si en desaccord fort
        if not same_direction:
            # Reduire le signal si desaccord
            base_signal *= 0.7
        
        return np.clip(base_signal, -1, 1)
    
    def compute_position_size(self, signal: float, confidence: float) -> float:
        """
        Calcule la taille de position basee sur le signal.
        """
        if abs(signal) < self.signal_threshold:
            return 0.0
        
        # Position size proportionnelle au signal et a la confiance
        base_size = abs(signal) * confidence
        
        # Scaling non-lineaire pour reduire les positions extremes
        scaled_size = np.tanh(base_size * 2) * 0.5  # Max 50% of portfolio
        
        return scaled_size
    
    def generate_signal(
        self,
        symbol: str,
        llm_signal: float,
        llm_confidence: float,
        technical_data: Dict
    ) -> HybridSignal:
        """
        Genere un signal hybride complet.
        """
        # Signal technique
        tech_signal, tech_confidence = self.compute_technical_signal(technical_data)
        
        # Fusion
        fused = self.fuse_signals(
            llm_signal, llm_confidence,
            tech_signal, tech_confidence
        )
        
        # Direction
        if fused > self.signal_threshold:
            direction = "BUY"
        elif fused < -self.signal_threshold:
            direction = "SELL"
        else:
            direction = "HOLD"
        
        # Position size
        avg_confidence = (llm_confidence + tech_confidence) / 2
        position_size = self.compute_position_size(fused, avg_confidence)
        
        return HybridSignal(
            symbol=symbol,
            llm_signal=llm_signal,
            llm_confidence=llm_confidence,
            technical_signal=tech_signal,
            technical_confidence=tech_confidence,
            fused_signal=fused,
            final_direction=direction,
            position_size=position_size
        )


# Demonstration
hybrid_system = HybridSignalSystem()

# Utiliser les resultats precedents
print("Systeme Hybride LLM + Technique")
print("="*60)

hybrid_signal = hybrid_system.generate_signal(
    symbol="AAPL",
    llm_signal=agg_signal,
    llm_confidence=agg_conf,
    technical_data=sample_technical
)

print(f"\nSignaux d'entree:")
print(f"  LLM Signal: {hybrid_signal.llm_signal:.2f} (conf: {hybrid_signal.llm_confidence:.2f})")
print(f"  Technical Signal: {hybrid_signal.technical_signal:.2f} (conf: {hybrid_signal.technical_confidence:.2f})")

print(f"\nSignal Fusionne:")
print(f"  Fused Signal: {hybrid_signal.fused_signal:.2f}")
print(f"  Direction: {hybrid_signal.final_direction}")
print(f"  Position Size: {hybrid_signal.position_size:.1%}")

In [ ]:
# Visualisation du systeme hybride

def visualize_hybrid_signals(signals: List[HybridSignal]):
    """
    Visualise les signaux hybrides.
    """
    if not signals:
        print("Pas de signaux a visualiser")
        return
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # Signal comparison
    ax1 = axes[0, 0]
    symbols = [s.symbol for s in signals]
    llm_signals = [s.llm_signal for s in signals]
    tech_signals = [s.technical_signal for s in signals]
    fused_signals = [s.fused_signal for s in signals]
    
    x = np.arange(len(symbols))
    width = 0.25
    
    ax1.bar(x - width, llm_signals, width, label='LLM', color='blue', alpha=0.7)
    ax1.bar(x, tech_signals, width, label='Technical', color='green', alpha=0.7)
    ax1.bar(x + width, fused_signals, width, label='Fused', color='red', alpha=0.7)
    
    ax1.set_xlabel('Symbol')
    ax1.set_ylabel('Signal Strength')
    ax1.set_title('Signal Comparison', fontsize=12, fontweight='bold')
    ax1.set_xticks(x)
    ax1.set_xticklabels(symbols)
    ax1.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Confidence levels
    ax2 = axes[0, 1]
    llm_conf = [s.llm_confidence for s in signals]
    tech_conf = [s.technical_confidence for s in signals]
    
    ax2.bar(x - width/2, llm_conf, width, label='LLM Confidence', color='blue', alpha=0.7)
    ax2.bar(x + width/2, tech_conf, width, label='Technical Confidence', color='green', alpha=0.7)
    
    ax2.set_xlabel('Symbol')
    ax2.set_ylabel('Confidence')
    ax2.set_title('Confidence Levels', fontsize=12, fontweight='bold')
    ax2.set_xticks(x)
    ax2.set_xticklabels(symbols)
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # Position sizes
    ax3 = axes[1, 0]
    positions = [s.position_size * 100 for s in signals]
    colors = ['green' if s.final_direction == 'BUY' else 'red' if s.final_direction == 'SELL' else 'gray' 
              for s in signals]
    
    ax3.bar(symbols, positions, color=colors, alpha=0.7)
    ax3.set_xlabel('Symbol')
    ax3.set_ylabel('Position Size (%)')
    ax3.set_title('Recommended Position Sizes', fontsize=12, fontweight='bold')
    ax3.grid(True, alpha=0.3)
    
    # Signal scatter
    ax4 = axes[1, 1]
    for s in signals:
        color = 'green' if s.final_direction == 'BUY' else 'red' if s.final_direction == 'SELL' else 'gray'
        ax4.scatter(s.llm_signal, s.technical_signal, s=200, c=color, alpha=0.7, 
                   label=f"{s.symbol} ({s.final_direction})")
    
    ax4.axhline(y=0, color='black', linestyle='--', alpha=0.3)
    ax4.axvline(x=0, color='black', linestyle='--', alpha=0.3)
    ax4.set_xlabel('LLM Signal')
    ax4.set_ylabel('Technical Signal')
    ax4.set_title('Signal Agreement', fontsize=12, fontweight='bold')
    ax4.set_xlim(-1.2, 1.2)
    ax4.set_ylim(-1.2, 1.2)
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()


# Generer quelques signaux de demonstration
demo_symbols = ['AAPL', 'MSFT', 'GOOGL', 'AMZN']
demo_signals = []

# Donnees simulees
demo_data = {
    'AAPL': {'price': 185, 'sma_20': 182, 'sma_50': 178, 'rsi_14': 62, 'macd': {'histogram': 0.3}, 'volume_ratio': 1.2},
    'MSFT': {'price': 380, 'sma_20': 385, 'sma_50': 375, 'rsi_14': 55, 'macd': {'histogram': -0.2}, 'volume_ratio': 0.9},
    'GOOGL': {'price': 140, 'sma_20': 138, 'sma_50': 142, 'rsi_14': 48, 'macd': {'histogram': 0.1}, 'volume_ratio': 1.0},
    'AMZN': {'price': 175, 'sma_20': 172, 'sma_50': 168, 'rsi_14': 72, 'macd': {'histogram': 0.5}, 'volume_ratio': 1.4},
}

demo_llm = {
    'AAPL': (0.6, 0.75),
    'MSFT': (0.2, 0.6),
    'GOOGL': (-0.3, 0.55),
    'AMZN': (0.5, 0.8),
}

for symbol in demo_symbols:
    llm_sig, llm_conf = demo_llm[symbol]
    signal = hybrid_system.generate_signal(
        symbol=symbol,
        llm_signal=llm_sig,
        llm_confidence=llm_conf,
        technical_data=demo_data[symbol]
    )
    demo_signals.append(signal)

visualize_hybrid_signals(demo_signals)

---

## Partie 6 : Integration QuantConnect (15 min)

### Architecture pour Production

```
EXTERNE (API Keys secrets)
        |
        v
QuantConnect Scheduled Event (Daily)
        |
        v
Fetch News (NewsAPI, TiingoNews)
        |
        v
LLM Analysis (OpenAI/Anthropic)
        |
        v
Signal Fusion
        |
        v
Alpha Model Insights
```

In [ ]:
# Code QuantConnect pour LLM Alpha Model

qc_llm_code = '''
from AlgorithmImports import *
import openai
import json
import os


class LLMTradingAlphaModel(AlphaModel):
    """
    Alpha Model utilisant les LLMs pour l'analyse de sentiment.
    
    Features:
    - Analyse quotidienne des news
    - Fusion avec indicateurs techniques
    - Gestion des couts API
    - Caching des resultats
    """
    
    # Prompt template
    SENTIMENT_PROMPT = """
Analyze the following news for {symbol} and provide a trading signal.

NEWS:
{news}

Respond in JSON format only:
{{"sentiment": "BULLISH"|"BEARISH"|"NEUTRAL", "confidence": 0.0-1.0, "reasoning": "brief explanation"}}
"""
    
    def __init__(
        self,
        openai_api_key: str,
        model: str = "gpt-4o-mini",
        max_daily_cost: float = 1.0,
        technical_weight: float = 0.4,
        llm_weight: float = 0.4
    ):
        self.api_key = openai_api_key
        self.model = model
        self.max_daily_cost = max_daily_cost
        self.technical_weight = technical_weight
        self.llm_weight = llm_weight
        
        # Tracking
        self.symbols = []
        self.last_analysis = {}
        self.daily_cost = 0.0
        self.last_cost_reset = datetime.min
        
        # Technical indicators
        self.indicators = {}
        
        # Initialize OpenAI client
        openai.api_key = self.api_key
    
    def Update(self, algorithm: QCAlgorithm, data: Slice) -> List[Insight]:
        insights = []
        
        # Reset daily cost
        if algorithm.Time.date() > self.last_cost_reset.date():
            self.daily_cost = 0.0
            self.last_cost_reset = algorithm.Time
        
        # Only run once per day
        if algorithm.Time.hour != 10:  # 10 AM
            return insights
        
        for symbol in self.symbols:
            if not data.ContainsKey(symbol):
                continue
            
            try:
                # Get news (using TiingoNews or custom data)
                news = self._get_news(algorithm, symbol)
                
                # Get LLM signal (if within budget)
                llm_signal, llm_confidence = self._get_llm_signal(
                    algorithm, symbol, news
                )
                
                # Get technical signal
                tech_signal, tech_confidence = self._get_technical_signal(
                    algorithm, symbol
                )
                
                # Fuse signals
                fused_signal = self._fuse_signals(
                    llm_signal, llm_confidence,
                    tech_signal, tech_confidence
                )
                
                # Generate insight if signal is strong enough
                if abs(fused_signal) > 0.3:
                    direction = InsightDirection.Up if fused_signal > 0 else InsightDirection.Down
                    confidence = min(abs(fused_signal), 1.0)
                    
                    insight = Insight.Price(
                        symbol,
                        timedelta(days=5),
                        direction,
                        magnitude=abs(fused_signal) * 0.02,
                        confidence=confidence,
                        sourceModel="LLM-Hybrid"
                    )
                    insights.append(insight)
                    
                    algorithm.Debug(
                        f"LLM Insight: {symbol} {direction.name} "
                        f"(signal: {fused_signal:.2f}, conf: {confidence:.2f})"
                    )
                    
            except Exception as e:
                algorithm.Debug(f"Error analyzing {symbol}: {e}")
        
        return insights
    
    def _get_news(self, algorithm: QCAlgorithm, symbol: Symbol) -> str:
        """
        Recupere les news recentes pour un symbole.
        """
        # Option 1: TiingoNews (premium)
        # news_data = algorithm.AddData(TiingoNews, symbol)
        
        # Option 2: Custom data source
        # Implementer selon votre source de donnees
        
        # Placeholder pour demonstration
        return f"Recent news about {symbol.Value}..."
    
    def _get_llm_signal(self, algorithm: QCAlgorithm, symbol: Symbol, news: str) -> tuple:
        """
        Obtient le signal LLM.
        """
        # Check budget
        estimated_cost = 0.001  # ~$0.001 per request for gpt-4o-mini
        if self.daily_cost + estimated_cost > self.max_daily_cost:
            algorithm.Debug("LLM budget exceeded, using cached/default")
            cached = self.last_analysis.get(str(symbol))
            if cached:
                return cached
            return 0.0, 0.5
        
        try:
            prompt = self.SENTIMENT_PROMPT.format(
                symbol=symbol.Value,
                news=news[:2000]
            )
            
            response = openai.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "user", "content": prompt}
                ],
                max_tokens=200,
                temperature=0.3
            )
            
            content = response.choices[0].message.content
            result = json.loads(content)
            
            # Update cost tracking
            self.daily_cost += estimated_cost
            
            # Parse result
            sentiment = result.get("sentiment", "NEUTRAL")
            confidence = result.get("confidence", 0.5)
            
            signal = {
                "BULLISH": 1.0,
                "BEARISH": -1.0,
                "NEUTRAL": 0.0
            }.get(sentiment, 0.0)
            
            # Cache result
            self.last_analysis[str(symbol)] = (signal * confidence, confidence)
            
            return signal * confidence, confidence
            
        except Exception as e:
            algorithm.Debug(f"LLM error: {e}")
            return 0.0, 0.5
    
    def _get_technical_signal(self, algorithm: QCAlgorithm, symbol: Symbol) -> tuple:
        """
        Calcule le signal technique.
        """
        if symbol not in self.indicators:
            return 0.0, 0.5
        
        ind = self.indicators[symbol]
        signals = []
        
        # SMA trend
        if ind['sma_fast'].IsReady and ind['sma_slow'].IsReady:
            if ind['sma_fast'].Current.Value > ind['sma_slow'].Current.Value:
                signals.append(1.0)
            else:
                signals.append(-1.0)
        
        # RSI
        if ind['rsi'].IsReady:
            rsi = ind['rsi'].Current.Value
            if rsi > 70:
                signals.append(-0.5)
            elif rsi < 30:
                signals.append(0.5)
        
        if not signals:
            return 0.0, 0.5
        
        signal = sum(signals) / len(signals)
        confidence = 0.6 + 0.2 * (len([s for s in signals if abs(s) > 0.3]) / len(signals))
        
        return signal, confidence
    
    def _fuse_signals(self, llm_signal, llm_conf, tech_signal, tech_conf) -> float:
        """
        Fusionne les signaux.
        """
        fused = (
            self.llm_weight * llm_signal * llm_conf +
            self.technical_weight * tech_signal * tech_conf
        )
        
        # Bonus for agreement
        if llm_signal * tech_signal > 0:
            fused *= 1.2
        
        return max(-1, min(1, fused))
    
    def OnSecuritiesChanged(self, algorithm: QCAlgorithm, changes: SecurityChanges):
        for security in changes.AddedSecurities:
            symbol = security.Symbol
            if symbol not in self.symbols:
                self.symbols.append(symbol)
                
                # Setup indicators
                self.indicators[symbol] = {
                    'sma_fast': algorithm.SMA(symbol, 20, Resolution.Daily),
                    'sma_slow': algorithm.SMA(symbol, 50, Resolution.Daily),
                    'rsi': algorithm.RSI(symbol, 14, Resolution.Daily)
                }
        
        for security in changes.RemovedSecurities:
            symbol = security.Symbol
            if symbol in self.symbols:
                self.symbols.remove(symbol)


class LLMTradingAlgorithm(QCAlgorithm):
    """
    Algorithme de trading utilisant les LLMs.
    """
    
    def Initialize(self):
        self.SetStartDate(2024, 1, 1)
        self.SetEndDate(2024, 12, 31)
        self.SetCash(100000)
        
        # API key from secrets
        openai_key = self.GetParameter("openai_api_key")
        
        # Universe
        self.AddUniverse(self.CoarseFilter)
        
        # Models
        self.SetAlpha(LLMTradingAlphaModel(
            openai_api_key=openai_key,
            model="gpt-4o-mini",
            max_daily_cost=1.0
        ))
        
        self.SetPortfolioConstruction(EqualWeightingPortfolioConstructionModel())
        self.SetExecution(ImmediateExecutionModel())
        self.SetRiskManagement(MaximumDrawdownPercentPerSecurity(0.05))
    
    def CoarseFilter(self, coarse):
        filtered = [x for x in coarse
                   if x.HasFundamentalData
                   and x.Price > 10
                   and x.DollarVolume > 10000000]
        return [x.Symbol for x in sorted(filtered, key=lambda x: x.DollarVolume, reverse=True)[:20]]
'''

print("LLMTradingAlgorithm code genere")
print("\nCaracteristiques:")
print("  - Analyse quotidienne avec GPT-4o-mini")
print("  - Budget API: $1/jour")
print("  - Fusion LLM + Technical")
print("  - Caching des resultats")

In [ ]:
# Resume et meilleures pratiques

print("="*70)
print("RESUME : LLM TRADING SIGNALS")
print("="*70)

best_practices = """
1. PROMPT ENGINEERING
   - Instructions specifiques et structurees
   - Format de sortie JSON
   - Exemples few-shot si necessaire
   - Temperature basse (0.2-0.4) pour coherence

2. GESTION DES COUTS
   - Budget quotidien strict
   - Modeles economiques (gpt-4o-mini, haiku)
   - Caching des resultats
   - Batching des requetes

3. ROBUSTESSE
   - Fallback vers signaux techniques
   - Validation du JSON
   - Retry avec backoff
   - Logging complet

4. SYSTEME HYBRIDE
   - Ne jamais se fier au LLM seul
   - Fusion avec indicateurs objectifs
   - Bonus pour accord LLM/technique
   - Penalite pour desaccord

5. PRODUCTION
   - API keys en secrets QuantConnect
   - Scheduled events quotidiens
   - Monitoring des couts en temps reel
   - Alertes sur erreurs API
"""

print(best_practices)

print("\nESTIMATION DES COUTS MENSUELS:")
print("  - 20 actifs x 1 analyse/jour x 30 jours = 600 requetes")
print("  - GPT-4o-mini: ~$3-5/mois")
print("  - GPT-4o: ~$15-25/mois")
print("  - Claude 3.5 Sonnet: ~$20-30/mois")

---

## Conclusion et Prochaines Etapes

### Recapitulatif

| Sujet | Points Cles |
|-------|-------------|
| **LLMs Finance** | Sentiment analysis, news interpretation, reasoning |
| **Prompt Engineering** | Structure, JSON output, few-shot, temperature |
| **GPT-4** | Excellent JSON, rapide, cout modere |
| **Claude** | Meilleur raisonnement, contexte long |
| **Systeme Hybride** | LLM + technique, fusion ponderee, robustesse |
| **Production** | Budget, caching, secrets, monitoring |

### Limitations et Risques

| Risque | Mitigation |
|--------|------------|
| **Hallucinations** | Validation croisee, signaux techniques |
| **Latence** | Caching, batch processing |
| **Couts** | Budget strict, modeles economiques |
| **Rate limits** | Retry, backoff, quotas |

### Ressources Complementaires

- [OpenAI API Documentation](https://platform.openai.com/docs)
- [Anthropic Claude API](https://docs.anthropic.com/)
- [Prompt Engineering Guide](https://www.promptingguide.ai/)
- [QuantConnect Alternative Data](https://www.quantconnect.com/docs/v2/writing-algorithms/datasets/alternative-data)

### Prochain Notebook

**QC-Py-27 - Production Deployment** : Paper trading, live trading, monitoring, et deploiement en production.

---

**Notebook complete. Vous maitrisez maintenant l'integration des LLMs pour le trading.**